<a href="https://colab.research.google.com/github/yhnx/EN3150_Assignment_03/blob/sineth_training/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
# =====================================================
# 3. Defining the CNN initally on an arbitary basis!!!
# ====================================================
class WasteCNN(nn.Module):
    def __init__(self, num_classes):
        super(WasteCNN, self).__init__()

        # Conv1: 3x3 kernel
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        # Conv2: 3x3 kernel with stride
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)

        # Conv3: 5x5 kernel
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(2, 2)

        # Fully connected
        self.fc1 = nn.Linear(64 * 16 * 16, 256)  # Adjust based on input image size
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool2(x)

        x = x.view(x.size(0), -1)  # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import numpy as np
EPOCHS = 80
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

In [ ]:
# ==== 4. Modular Training Function ====
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, device='gpu'):
    train_losses, val_losses, val_accs = [], [], []

    for epoch in range(num_epochs):
        # ---- Training ----
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # ---- Validation ----
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_acc = 100 * correct / total
        val_accs.append(val_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"| Train Loss: {avg_train_loss:.4f} "
              f"| Val Loss: {avg_val_loss:.4f} "
              f"| Val Acc: {val_acc:.2f}%")

    # ---- Return metrics ----
    return train_losses, val_losses, val_accs


In [ ]:
# ==== 5. Evaluation Function ====
def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    correct, total = 0, 0
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    acc = 100 * correct / total
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    cm = confusion_matrix(y_true, y_pred)

    print(f"\nTest Accuracy: {acc:.2f}%")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print("Confusion Matrix:\n", cm)

    return acc, precision, recall, f1, cm


In [ ]:
# ==== 6. Plotting Function ====
def plot_loss_curves(train_losses, val_losses, title="Training and Validation Loss"):
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(title)
    plt.show()

In [ ]:
# ==== 7. Running Experiments ====
num_classes = len(dataset.classes)
criterion = nn.CrossEntropyLoss()

# Different optimizers to compare
optimizers = {
    "Adam": optim.Adam,
    "SGD": optim.SGD,
    "SGD+Momentum": lambda params, lr: optim.SGD(params, lr=lr, momentum=0.9)
}

results = {}

for opt_name, opt_fn in optimizers.items():
    print(f"\n===== Training with {opt_name} Optimizer =====")
    model = CustomCNN(num_classes=num_classes).to(DEVICE)
    optimizer = opt_fn(model.parameters(), lr=LEARNING_RATE)

    train_losses, val_losses, val_accs = train_model(
        model, train_loader, val_loader, criterion, optimizer,
        num_epochs=EPOCHS, device=DEVICE
    )

    test_metrics = evaluate_model(model, test_loader, device=DEVICE)

    results[opt_name] = {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "val_accs": val_accs,
        "test_metrics": test_metrics
    }

    plot_loss_curves(train_losses, val_losses, title=f"Loss Curves - {opt_name}")